[Герман Гаврилов](https://drive.google.com/drive/folders/17-r9k1eEGHcVXzGk87dFqfR6FlNfFAYL?usp=sharing) - моя рабочая папка.

# 3 неделя

In [ ]:
!rm -r sample_data

In [ ]:
# Подключаем google-диск
from google.colab import drive
drive.mount('/content/drive')
my_drv_path = '/content/drive/MyDrive/'

Mounted at /content/drive


## Библиотеки

In [ ]:
# Установка cohere, openai и tiktoken
!pip -q install cohere openai==1.2.3 tiktoken==0.5.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.7/51.7 kB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.3/220.3 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 34.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 65.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.1 MB/s eta 0:00:00


In [ ]:
import os
import re
import shutil
import openai
import getpass
import requests
from IPython.display import HTML, display

## Переменные

In [ ]:
GPT4_MODEL  = 'gpt-4-1106-preview'
GPT35_MODEL = 'gpt-3.5-turbo-1106'
separator   = '\n================================================\n'

## Функции

In [ ]:
# Функция для переноса текста в output ячейках
def set_css():

  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))

get_ipython().events.register('pre_run_cell', set_css)


# Progressbar
def progress(value, max=100):

  return HTML('''
      <progress
          value='{value}'
          max='{max}',
          style='width: 100%'
      >
          {value}
      </progress>
  '''.format(value=value, max=max))


# Функция для загрузки документа по ссылке из гугл драйв
def load_document_text(url: str) -> str:

  # Extract the document ID from the URL
  match_ = re.search('/document/d/([a-zA-Z0-9-_]+)', url)
  if match_ is None:
    raise ValueError('Invalid Google Docs URL')
  doc_id = match_.group(1)

  # Download the document as plain text
  response = requests.get(f'https://docs.google.com/document/d/{doc_id}/export?format=txt')
  response.raise_for_status()
  text = response.text

  return text


# Функция для загрузки документа из файла по ссылке из гугл драйв
def load_document_text_from_file(url: str) -> str:

  # Extract the document ID from the URL
  match_ = re.search('/file/d/([a-zA-Z0-9-_]+)', url)

  if match_ is None:
    raise ValueError('Invalid File URL')
  file_id = match_.group(1)

  # Download the document from the file
  response = requests.get(f'https://drive.google.com/uc?id={file_id}')
  content = response.content.decode('utf-8')

  return content


# Функция для сохранения текста в файл
def write_text_to_file(file_path, text):

  with open(file_path, 'w') as f:
    f.write(text)


# Функция для чтения текста из файла
def read_text_from_file(file_path):

  with open(file_path, 'r') as f:
    content = f.read()
    return(content)


# Функция для ввода OpenAI API-ключа
def get_key_ОpenAI():

  openai.api_key = getpass.getpass('OpenAI API Key:')
  os.environ['OPENAI_API_KEY'] = openai.api_key


# Функция для получения ответа chatGPT
def get_gpt_response(system, context, instructions, temp=0, model='gpt-3.5-turbo-1106'):

  messages = [{'role': 'system', 'content': system}, {'role': 'user', 'content': f'{instructions}{context}'}]
  completion = openai.chat.completions.create(model=model, messages=messages, temperature=temp)

  return completion.choices[0].message.content

# Функция для обработки постов
def analyze_posts(posts, system, instructions, posts_amount=10):

  results = []
  max = len(posts)
  out = display(progress(0, max), display_id=True)

  for i in range(0, max, posts_amount):

    # Объединяем посты в один
    combined_posts = '\n\n'.join(posts[i:i+posts_amount])

    response = get_gpt_response(system, combined_posts, instructions)
    results.append(response)
    out.update(progress(i, max-posts_amount))

  return results

## Ввод `OpenAI API-ключа`


In [ ]:
get_key_ОpenAI()

OpenAI API Key:··········


## Выделение стилистических особенностей из постов

In [ ]:
# Путь для файлов стилистических особенностей из постов
stylistic_features_path = my_drv_path + 'Герман Гаврилов' + '/Стилистические особенности из постов/'

In [ ]:
# Создаём папку для файлов стилистических особенностей из постов
os.makedirs(stylistic_features_path, exist_ok=True)

In [ ]:
#@title Загрузка постов
posts_file_url = 'https://docs.google.com/document/d/1u01H0WKjgYDJV2-JtLm_Kgd-1hGRKTOhwCv935IFcsk/'
full_document = load_document_text(posts_file_url)
posts = full_document.split('##_')
len(posts)

210

### Выделение типичных конструкций предложений, синонимичных выражений и фраз

In [ ]:
system = """
Вы - первоклассный литературный критик. Ваша задача определить особенности, которые составляют уникальный стиль автора,
для последующего использования этих особенностей для генерации постов в стиле автора. Если вы хорошо справитесь с задачей, вы получите щедрые чаевые.
Результатом вашей работы должен быть список выражений и фраз, характерных для автора.
"""

instructions = """
Я передам вам несколько постов автора.
Анализируйте особенности используемых автором словосочетаний и фраз.
Обратите внимание на типичные конструкции предложений, синонимичные выражения и фразы, которые он предпочитает использовать. Приведите примеры.
Тексты постов для анализа:\n
"""

results = analyze_posts(posts, system, instructions)
write_text_to_file(stylistic_features_path + 'phrases.txt', separator.join(results))

In [ ]:
!cat '/content/drive/MyDrive/Герман Гаврилов/Стилистические особенности из постов/phrases.txt'

Автор текстов обладает уникальным стилем, который можно охарактеризовать следующими особенностями:

1. Использование эмодзи и символов: Автор часто использует эмодзи и символы, чтобы выразить свои эмоции и отношение к описываемым событиям. Например, использование символов 📲, 📉, 🤠, 👻, 🙄, 🙈, 😊, 🤬, 😎, 🤑, 🙏, 🤠, 🤑, 🙈, 🙏, 🤑, 🙈, 🙏, 🤑, 🙈, 🙏, 🤑, 🙈, 🙏, 🤑, 🙈, 🙏, 🤑, 🙈, 🙏, 🤑, 🙈, 🙏, 🤑, 🙈, 🙏, 🤑, 🙈, 🙏, 🤑, 🙈, 🙏, 🤑, 🙈, 🙏, 🤑, 🙈, 🙏, 🤑, 🙈, 🙏, 🤑, 🙈, 🙏, 🤑, 🙈, 🙏, 🤑, 🙈, 🙏, 🤑, 🙈, 🙏, 🤑, 🙈, 🙏, 🤑, 🙈, 🙏, 🤑, 🙈, 🙏, 🤑, 🙈, 🙏, 🤑, 🙈, 🙏, 🤑, 🙈, 🙏, 🤑, 🙈, 🙏, 🤑, 🙈, 🙏, 🤑, 🙈, 🙏, 🤑, 🙈, 🙏, 🤑, 🙈, 🙏, 🤑, 🙈, 🙏, 🤑, 🙈, 🙏, 🤑, 🙈, 🙏, 🤑, 🙈, 🙏, 🤑, 🙈, 🙏, 🤑, 🙈, 🙏, 🤑, 🙈, 🙏, 🤑, 🙈, 🙏, 🤑, 🙈, 🙏, 🤑, 🙈, 🙏, 🤑, 🙈, 🙏, 🤑, 🙈, 🙏, 🤑, 🙈, 🙏, 🤑, 🙈, 🙏, 🤑, 🙈, 🙏, 🤑, 🙈, 🙏, 🤑, 🙈, 🙏, 🤑, 🙈, 🙏, 🤑, 🙈, 🙏, 🤑, 🙈, 🙏, 🤑, 🙈, 🙏, 🤑, 🙈, 🙏, 🤑, 🙈, 🙏, 🤑, 🙈, 🙏, 🤑, 🙈, 🙏, 🤑, 🙈, 🙏, 🤑, 🙈, 🙏, 🤑, 🙈, 🙏, 🤑, 🙈, 🙏, 🤑, 🙈, 🙏, 🤑, 🙈, 🙏, 🤑, 🙈, 🙏, 🤑, 🙈, 🙏, 🤑, 🙈, 🙏, 🤑, 🙈, 🙏, 🤑, 🙈, 🙏, 🤑, 🙈, 🙏, 🤑, 🙈, 🙏, 🤑, 🙈, 🙏, 🤑, 🙈, 🙏, 🤑, 🙈, 🙏, 🤑, 🙈, 🙏, 🤑, 🙈, 🙏, 🤑, 🙈, 🙏, 🤑, 🙈, 🙏, 🤑, 🙈, 🙏, 🤑, 🙈, 🙏, 🤑, 🙈, 🙏, 🤑, 🙈, 🙏

In [ ]:
shutil.move(stylistic_features_path + 'phrases.txt', stylistic_features_path + 'phrases_by_me.txt')

'/content/drive/MyDrive/Герман Гаврилов/Стилистические особенности из постов/phrases_by_me.txt'

In [ ]:
!cat '/content/drive/MyDrive/Герман Гаврилов/Стилистические особенности из постов/phrases.txt'

Исходя из предоставленных постов, можно выделить несколько особенностей стиля автора:

1. Использование эмодзи и символов: автор часто использует эмодзи, чтобы выразить свои эмоции и отношение к описываемым событиям. Например, эмодзи 👻, 📲, 📉, 🕺, 🤠, 🙄, 😊, 🤬, 🙈, 🙏, 🤑, 🤠, 🙄, 😎, 🤑, 🙏, 🙈 используются для передачи эмоциональной окраски текста.

2. Использование неформальной лексики: автор предпочитает использовать разговорный стиль, включая фразы типа "что-то удивило тебя?", "как и для меня, думаю большинству подходит первый пункт", "Не сравнивай месяц с месяцем", что создает близость и доверие к аудитории.

3. Использование аббревиатур и хештегов: автор активно использует хештеги для обозначения темы поста, такие как #инвестиции, #IPO, #новости, #управление. Также присутствуют аббревиатуры, такие как IPO (Initial Public Offering), B2B (Business to Business), EdTech (Educational Technology).

4. Использование примеров и личного опыта: автор часто приводит примеры из собственной практики, что

### Выделение эмоциональной окраски

In [ ]:
system = """
Вы - первоклассный литературный критик. Ваша задача определить особенности которые составляют уникальный стиль автора,
для последующего использования этих особенностей для генерации постов в стиле автора. Если вы хорошо справитесь с задачей вы получите щедрые чаевые.
"""

instructions = """
Я передам вам несколько постов автора.
Определите общий тон и настроение, которое автор поддерживает в своих постах.
Обращайте внимание на используемую эмоциональную окраску, характеристику событий или феноменов, а также соотношение между позитивными и негативными выражениями.
Приведите примеры.
Тексты постов для анализа:\n
"""

results = analyze_posts(posts, system, instructions)
write_text_to_file(stylistic_features_path + 'tones.txt', separator.join(results))

In [ ]:
!cat '/content/drive/MyDrive/Герман Гаврилов/Стилистические особенности из постов/tones.txt'

Автор поддерживает в своих постах нейтральный и информативный тон, смешанный с нотками юмора. Он делится своим опытом в инвестировании, предостерегает от возможных ошибок и рисков, а также дает советы по участию в IPO. В его постах преобладает позитивная эмоциональная окраска, даже когда речь идет о потенциальных рисках. Он также задает вопросы, чтобы вызвать обсуждение и поделиться новостями из мира инвестиций.

Примеры:
1. В посте "Мой опыт инвестиций" автор демонстрирует свой опыт инвестирования, описывает свои решения и эмоции, используя эмодзи и юмористические обороты. Он дает советы, но при этом предостерегает от возможных ошибок, сохраняя легкость и непринужденность в выражении.

2. В посте "Инвест новости прошедшей недели" автор представляет новости из мира инвестиций, при этом призывая читателя задуматься и поделиться своим мнением. Он использует информативный и нейтральный тон, делится интересными фактами, но при этом задает вопрос, чтобы вызвать реакцию у аудитории.

3. В по

### Выделение основных тем, которые обсуждает автор в своих постах

In [ ]:
system = """
Вы - первоклассный литературный критик. Ваша задача определить особенности которые составляют уникальный стиль автора,
для последующего использования этих особенностей для генерации постов в стиле автора. Если вы хорошо справитесь с задачей вы получите щедрые чаевые.
"""

instructions = """
Я передам вам несколько постов автора.
Узнайте основные темы, которые интересуют автора, и на которые он фокусируется в своих постах.
Исследуйте, как он подходит к обсуждению этих тем и какие ключевые фразы или референсы он использует в своем контенте.
Приведите примеры.
Тексты постов для анализа:\n
"""

results = analyze_posts(posts, system, instructions)
write_text_to_file(stylistic_features_path + 'themes.txt', separator.join(results))

In [ ]:
!cat '/content/drive/MyDrive/Герман Гаврилов/Стилистические особенности из постов/themes.txt'

Автор фокусируется на теме инвестиций, IPO и управлении финансами. Он демонстрирует свой опыт в инвестировании, делясь личными историями успеха и неудач. Он также обсуждает новости и события в мире инвестиций, а также дает советы по участию в IPO. 

Автор использует разговорный стиль, эмодзи и шутки, чтобы сделать свои посты более доступными и легкими для восприятия. Он также часто обращается к своему личному опыту, чтобы подкрепить свои советы и рекомендации.

Примеры ключевых фраз и референсов:
1. "Мой опыт инвестиций", "как классно инвестировать в IPO", "мега happy", "фоточки выше, кружками на графике отмечены мои выходы" - автор демонстрирует личный опыт и успехи в инвестировании.
2. "500% доходность на IPO", "как получить большую аллокацию?" - автор обсуждает тему IPO, демонстрируя свой опыт и делясь советами.
3. "Не сравнивай месяц с месяцем", "Средняя выручка в рабочий день и выходной день" - автор использует аналитический подход к управлению финансами и дает советы по управлени

### Выделение логических структур аргументации и стиля изложения

In [ ]:
system = """
Вы - первоклассный литературный критик. Ваша задача определить особенности которые составляют уникальный стиль автора,
для последующего использования этих особенностей для генерации постов в стиле автора. Если вы хорошо справитесь с задачей вы получите щедрые чаевые.
"""

instructions = """
Я передам вам несколько постов автора.
Изучайте методы, которыми автор поддерживает свои аргументы или позиции.
Обратите внимание на логические структуры, используемые аргументы и стиль изложения текста.
Приведите примеры.
Тексты постов для анализа:\n
"""

results = analyze_posts(posts, system, instructions)
write_text_to_file(stylistic_features_path + 'arguments.txt', separator.join(results))

In [ ]:
!cat '/content/drive/MyDrive/Герман Гаврилов/Стилистические особенности из постов/arguments.txt'

Автор обладает уникальным стилем, который отличается следующими особенностями:

1. Личный опыт: Автор часто обращается к собственному опыту, делясь своими инвестиционными решениями, успехами и неудачами. Он подчеркивает, что его советы основаны исключительно на личном опыте, что создает близость с аудиторией и делает его тексты более убедительными.

2. Использование примеров: Автор часто приводит конкретные примеры, скриншоты и графики, чтобы иллюстрировать свои рассуждения. Это делает его посты более наглядными и понятными для читателей.

3. Использование эмодзи: Автор активно использует эмодзи, чтобы выразить свои эмоции и отношение к обсуждаемым темам. Это придает его текстам индивидуальность и делает их более привлекательными для молодой аудитории.

4. Поддержка аргументов: Автор часто поддерживает свои рассуждения конкретными фактами, статистикой и ссылками на авторитетные источники. Это придает его постам авторитетность и убедительность.

5. Использование хештегов: Автор активно 

### Выделение особенностей пунктуации и грамматики

In [ ]:
system = """
Вы - первоклассный литературный критик. Ваша задача определить особенности которые составляют уникальный стиль автора,
для последующего использования этих особенностей для генерации постов в стиле автора. Если вы хорошо справитесь с задачей вы получите щедрые чаевые.
"""

instructions = """
Я передам вам несколько постов автора.
Отметьте особенности грамматики и пунктуации, которые характерны для автора.
Некоторые люди предпочитают использовать определенные формы предложений или имеют определенный стиль пунктуации, которые могут помочь вам эмулировать их стиль.
Приведите примеры.
Тексты постов для анализа:\n
"""

results = analyze_posts(posts, system, instructions)
write_text_to_file(stylistic_features_path + 'punctuation.txt', separator.join(results))

In [ ]:
!cat '/content/drive/MyDrive/Герман Гаврилов/Стилистические особенности из постов/punctuation.txt'

Автор обладает уникальным стилем, который отражается в его постах. Вот основные особенности грамматики и пунктуации, которые характерны для него:

1. Использование эмодзи: Автор часто использует эмодзи для выражения своих эмоций и отношения к теме. Например, он использует эмодзи 🙄, 🙈, 🤠, 🤑, 🤬, 😊, 📲, 📉, 📈, 🦠, и другие, чтобы добавить выразительности к тексту.

2. Использование хештегов: Автор активно использует хештеги для категоризации своих постов и привлечения внимания к ключевым темам. Например, #инвестиции, #IPO, #новости, #управление и другие.

3. Нестандартное использование знаков препинания: Автор часто использует знаки препинания, такие как восклицательные знаки, чтобы подчеркнуть важность или эмоциональную окраску высказывания. Он также использует многоточия для создания пауз и подчеркивания размышлений.

4. Использование аббревиатур и сокращений: Автор часто использует сокращения, такие как IPO, B2B, и другие, что придает его текстам более профессиональный и экспертный вид.



### Выявление уровня формальности

In [ ]:
system = """
Вы - первоклассный литературный критик. Ваша задача определить особенности которые составляют уникальный стиль автора,
для последующего использования этих особенностей для генерации постов в стиле автора. Если вы хорошо справитесь с задачей вы получите щедрые чаевые.
"""

instructions = """
Я передам вам несколько постов автора.
Разберитесь, как автор обращается к своей аудитории и какие приемы он использует, чтобы установить контакт с ней.
Обратите внимание на уровень формальности или неформальности, использование личных местоимений и другие элементы коммуникации с аудиторией.
Приведите примеры.
Тексты постов для анализа:\n
"""

results = analyze_posts(posts, system, instructions)
write_text_to_file(stylistic_features_path + 'formality.txt', separator.join(results))

In [ ]:
!cat '/content/drive/MyDrive/Герман Гаврилов/Стилистические особенности из постов/formality.txt'

Автор обращается к своей аудитории весьма неформально, используя личные местоимения, эмодзи, а также обращения "ты" и "вы". Он также демонстрирует свою личную позицию и опыт, призывая читателей делиться своим мнением. Автор часто использует примеры из своей жизни, чтобы проиллюстрировать свои мысли и советы.

Пример 1:
"Мой опыт инвестиций... Можете корректировать меня и конечно же не стоит пользоваться моим опытом и советами... Я продал почти все акции в сентябре 2020... И мне тоже захотелось попробовать."

Пример 2:
"Что понравилось или удивило тебя?"

Пример 3:
"Когда вы видите скриншотики «я вошёл в IPO и вот уже рост 500%» возможно вы представили, как человек заработал хренову тучу денег и едет покупать новую феррари."

Автор также использует примеры и иллюстрации, чтобы объяснить сложные концепции, делая их более доступными для аудитории.

Пример 4:
"Перед IPO собираются заявки... Если заявок пришло на 10ккк, а нужно 1ккк, то очевидно, что 9ккк получат отказ... В ней есть много н

### Выявление фирменных особенностей автора

In [ ]:
system = """
Вы - первоклассный литературный критик. Ваша задача определить особенности которые составляют уникальный стиль автора,
для последующего использования этих особенностей для генерации постов в стиле автора. Если вы хорошо справитесь с задачей вы получите щедрые чаевые.
"""

instructions = """
Я передам вам несколько постов автора.
Узнайте о каких-либо уникальных особенностях или стилевых изысках, которые отличают автора от других.
Это может быть что-то вроде фирменной фразы или повторяющегося мотива, который добавляет индивидуальность к его стилю письма.
Приведите примеры.
Тексты постов для анализа:\n
"""

results = analyze_posts(posts, system, instructions)
write_text_to_file(stylistic_features_path + 'features.txt', separator.join(results))

In [ ]:
!cat '/content/drive/MyDrive/Герман Гаврилов/Стилистические особенности из постов/features.txt'

Автор обладает несколькими уникальными стилистическими чертами, которые делают его посты узнаваемыми.

1. Использование эмодзи и символов. Автор часто использует эмодзи, чтобы выразить свои эмоции и отношение к теме. Например, он использует 👻, 📲, 📉, 🕺, 🦠, 🤠, 🙄, 🤬, 😊, 🙈, 🤑, 🙏, 😃, 🤎, и другие, чтобы добавить индивидуальность и эмоциональную окраску к своим постам.

2. Личный опыт и истории. Автор часто делился своими личными инвестиционными и бизнес-опытами, что придает его постам аутентичность и делает их более привлекательными для читателей.

3. Использование хештегов. Автор активно использует хештеги, чтобы помечать свои посты и делиться информацией на определенные темы, такие как #инвестиции, #IPO, #новости, #управление. Это помогает читателям быстро понять о чем пост и найти похожие темы.

4. Использование терминов и специфической лексики. Автор использует специфическую лексику, связанную с инвестициями и бизнесом, что создает впечатление экспертности и авторитетности.

5. Вопросы к

### Выявление особенностей форматирования постов

In [ ]:
system = """
Вы - первоклассный литературный критик. Ваша задача определить особенности которые составляют уникальный стиль автора,
для последующего использования этих особенностей для генерации постов в стиле автора. Если вы хорошо справитесь с задачей вы получите щедрые чаевые.
"""

instructions = """
Я передам вам несколько постов автора.
Анализируйте, как автор организует свои тексты: абзацы, заголовки, выделение ключевых фраз жирным шрифтом
Приведите примеры.
Тексты постов для анализа:\n
"""

results = analyze_posts(posts, system, instructions)
write_text_to_file(stylistic_features_path + 'formatting.txt', separator.join(results))

In [ ]:
!cat '/content/drive/MyDrive/Герман Гаврилов/Стилистические особенности из постов/formatting.txt'

Автор организует свои тексты следующим образом:

1. Использование эмодзи и хэштегов: Автор активно использует эмодзи и хэштеги для выделения ключевых тем и тэгов, что делает текст более ярким и привлекательным для читателей.

2. Заголовки: Автор часто начинает свои посты с заголовков, которые являются краткими и информативными, привлекая внимание к основным темам поста.

3. Абзацы: Текст разделен на небольшие абзацы, что делает его более удобным для чтения и понимания.

4. Выделение ключевых фраз: Автор использует выделение ключевых фраз жирным шрифтом для подчеркивания важности определенных моментов и идей.

Примеры:

1. В посте "Мой опыт инвестиций" автор использует эмодзи и хэштеги для привлечения внимания к теме инвестиций. Текст разделен на небольшие абзацы, и ключевые фразы выделены жирным шрифтом для подчеркивания важности определенных моментов.

2. В посте "Инвест новости прошедшей недели" автор использует нумерованный список для представления новостей, что делает информацию бо

## Получение частей промпта по каждому стилистическому аспекту

In [ ]:
#@title Ссылки на отчёты
phrases_report_url     = 'https://drive.google.com/file/d/1yadBlinF6myFYxGbYp_JzJdoI-FxsPZZ/'
tones_report_url       = 'https://drive.google.com/file/d/1OZG58pJVuzaLM5k3zqSXps2rrkNHQdtE/'
themes_report_url      = 'https://drive.google.com/file/d/1-1cMe5GNOzbh2msfYeJm4uaweIklqMtk/'
arguments_report_url   = 'https://drive.google.com/file/d/1SZXMugPUZZkFYUwrat7H_wipXI40ibBb/'
punctuation_report_url = 'https://drive.google.com/file/d/1-4Mffq2b7pcK8fH-4rJbKKe6q6vEMYny/'
formality_report_url   = 'https://drive.google.com/file/d/1-Aq1na9pC52hgs65hWuNtS6ELTXQF1ps/'
feature_report_urls    = 'https://drive.google.com/file/d/1-BfxpMed6TSQj3oFNJEmto8451U9NO5h/'
formatting_report_url  = 'https://drive.google.com/file/d/1-FsKwciGeNAMdJPcq0TQfrsAEMEcKw66/'

In [ ]:
system = """
Вы - первоклассный генератор промптов.
Ваша задача - создать часть одного большого промпта для ChatGPT, который будет создавать посты в Телеграм, четко следуя стилю автора.
Часть большого промпта должна отвечать за один из аспектов стиля автора.
"""

In [ ]:
#@title Фразы и выражения
report = load_document_text_from_file(phrases_report_url)
instructions = """
Я передам вам отчет по выделению из постов автора типичных конструкций выражений и фраз с примерами.
Вы помните, что отчеты сформированы на основе большого количества постов, и в вашем промте нужно сохранить как можно больше информации из переданных отчетов.
На основе переданных данных напишите часть промпта, которая будет отвечать за словосочетания и фразы.
Отчеты по анализу постов:\n
"""
result = get_gpt_response(system, report, instructions)
print(result)

Бл


In [ ]:
#@title Эмоциональная окраска
report = load_document_text_from_file(tones_report_url)
instructions = """
Я передам вам отчет по выделению из постов автора эмоциональной окраски с примерами.
Вы помните, что отчеты сформированы на основе большого количества постов, и в вашем промте нужно сохранить как можно больше информации из переданных отчетов.
На основе переданных данных напишите часть промпта, которая будет отвечать за общий тон и настроение, которое автор поддерживает в своих постах.
Отчеты по анализу постов:\n
"""
result = get_gpt_response(system, report, instructions)
print(result)

Чтобы отразить стиль автора в промпте для ChatGPT, я предлагаю следующую формулировку:

"Автор этих постов обладает уникальным стилем, который отличается прагматичностью, глубокими знаниями в области управления и бизнеса, а также эмоциональной отзывчивостью. Он стремится к практической пользе своих постов, предлагая конкретные советы и инструменты для управления бизнесом, развития личности и достижения успеха. В своих текстах автор использует яркие эмоциональные слова, чтобы подчеркнуть важность своих идей, а также обращается к своему личному опыту и примерам из жизни, чтобы подкрепить свои утверждения. Он также активно взаимодействует с аудиторией, задавая вопросы и призывая к обсуждению в комментариях. Таким образом, стиль автора можно охарактеризовать как прагматичный, эмоционально окрашенный, личный и интерактивный."

Этот текст отражает основные черты стиля автора, подчеркивая его экспертность, эмоциональную отзывчивость и вовлеченность в диалог с аудиторией.


In [ ]:
#@title Основные темы
report = load_document_text_from_file(themes_report_url)
instructions = """
Я передам вам отчет по выделению из постов автора основных тем, которые автор обсуждает в своих постах.
Вы помните, что отчеты сформированы на основе большого количества постов, и в вашем промте нужно сохранить как можно больше информации из переданных отчетов.
На основе переданных данных напишите часть промпта, которая будет отвечать за темы, которые автор обсуждает в своих постах.
Отчеты по анализу постов:\n
"""
result = get_gpt_response(system, report, instructions)
print(result)

Благодарю за предоставленную информацию. На основе переданных данных я могу сформулировать следующую часть промпта, отражающую стиль автора:

"Привет, друзья! Сегодня хочу поделиться с вами своими мыслями о том, как создать успешный бизнес, который будет приносить не только прибыль, но и радость от работы. Мой опыт управления компаниями и личного развития позволяет мне утверждать, что ключевыми факторами успеха являются систематизация бизнес-процессов, эффективное управление персоналом и постоянное личное развитие.

Когда я начинал свой путь, я часто сталкивался с трудностями, но каждая ошибка стала для меня уроком. Я убежден, что важно не зацикливаться на ошибках, а извлекать из них уроки и корректировать свой путь. Этот подход помог мне развивать свои компании и личные навыки.

Одним из ключевых моментов в моем опыте стало понимание важности системного подхода к управлению бизнесом. Я убежден, что систематизация, автоматизация и упрощение бизнес-процессов позволяют минимизировать зав

In [ ]:
#@title Аргументация
report = load_document_text_from_file(arguments_report_url)
instructions = """
Я передам вам отчет по выделению из постов автора логических структур аргументации и стиля изложения.
Вы помните, что отчеты сформированы на основе большого количества постов, и в вашем промте нужно сохранить как можно больше информации из переданных отчетов.
На основе переданных данных напишите часть промпта, которая будет отвечать за способы, которыми автор поддерживает свои аргументы или позиции.
Отчеты по анализу постов:\n
"""
result = get_gpt_response(system, report, instructions)
print(result)

Благодарю за предоставленные отчеты. На основе анализа стиля автора, можно выделить несколько ключевых особенностей, которые определяют его уникальный подход к поддержке аргументов и позиций:

1. Личный опыт: Автор часто обращается к собственному опыту, делясь инвестиционными решениями, успехами и неудачами. Он подчеркивает, что его советы основаны исключительно на личном опыте, что создает близость с аудиторией и делает его тексты более убедительными.

2. Использование примеров: Автор часто приводит конкретные примеры, скриншоты и графики, чтобы иллюстрировать свои рассуждения. Это делает его посты более наглядными и понятными для читателей.

3. Использование эмодзи: Автор активно использует эмодзи, чтобы выразить свои эмоции и отношение к обсуждаемым темам. Это придает его текстам индивидуальность и делает их более привлекательными для молодой аудитории.

4. Поддержка аргументов: Автор часто поддерживает свои рассуждения конкретными фактами, статистикой и ссылками на авторитетные ист

In [ ]:
#@title Пунктуация и граматика
report = load_document_text_from_file(punctuation_report_url)
instructions = """
Я передам вам отчет по выделению из постов автора особенностей пунктуации и граматики.
Вы помните, что отчеты сформированы на основе большого количества постов, и в вашем промте нужно сохранить как можно больше информации из переданных отчетов.
На основе переданных данных напишите часть промпта, которая будет отвечать за особенности грамматики и пунктуации, которые характерны для автора.
Отчеты по анализу постов:\n
"""
result = get_gpt_response(system, report, instructions)
print(result)

BadRequestError: ignored

In [ ]:
#@title Уровень формальности
report = load_document_text_from_file(formality_report_url)
instructions = """
Я передам вам отчет по выделению из постов уровня формальности.
Вы помните, что отчеты сформированы на основе большого количества постов, и в вашем промпте нужно сохранить как можно больше информации из переданных отчетов.
На основе переданных данных напишите часть промпта, которая будет отвечать за то, как автор обращается к своей аудитории и какие приемы он использует, чтобы установить контакт с ней.
Обратите внимание на уровень формальности или неформальности, использование личных местоимений и другие элементы коммуникации с аудиторией.
Отчеты по анализу постов:\n
"""
result = get_gpt_response(system, report, instructions)
print(result)

Отчет по анализу стиля автора позволяет выделить несколько ключевых особенностей его коммуникации с аудиторией:

1. Неформальный тон общения: Автор обращается к своей аудитории в дружелюбной и непринужденной манере, используя разговорный язык и личные местоимения, такие как "я" и "мы".

2. Использование эмодзи и символов: Частое использование эмодзи и символов добавляет эмоциональную окраску к текстам автора, делая их более привлекательными и интересными для аудитории.

3. Вовлечение аудитории: Автор активно вовлекает свою аудиторию, задавая вопросы, призывая к обсуждению и даже приглашая читателей поделиться своим опытом.

4. Использование личных примеров и иллюстраций: Автор часто использует личные примеры и иллюстрации, чтобы проиллюстрировать свои мысли и сделать материал более доступным для аудитории.

5. Практические советы и рекомендации: Автор предлагает конкретные советы и рекомендации, делая свои посты полезными и применимыми в реальной жизни.

Исходя из этого анализа, для со

In [ ]:
#@title Фишки автора
report = load_document_text_from_file(feature_report_urls)
instructions = """
Я передам вам отчет по выделению из постов особенных фишек автора.
Вы помните, что отчеты сформированы на основе большого количества постов, и в вашем промпте нужно сохранить как можно больше информации из переданных отчетов.
На основе переданных данных напишите часть промпта, которая будет отвечать за уникальные особенности или стилевые изыски, которые отличают автора от других.
Это может быть что-то вроде фирменной фразы или повторяющегося мотива, который добавляет индивидуальность к его стилю письма.
Отчеты по анализу постов:\n
"""
result = get_gpt_response(system, report, instructions)
print(result)

Отчет по выделению уникальных стилистических черт автора позволяет выделить несколько ключевых особенностей, которые делают его тексты узнаваемыми и привлекательными для аудитории:

1. Использование эмодзи и символов: Автор часто встраивает эмодзи в свои посты, чтобы подчеркнуть эмоциональную окраску высказываний и установить более доверительную атмосферу с читателями.

2. Личный опыт и истории: Автор часто демонстрирует свой личный опыт и успехи, чтобы вдохновить и поделиться знаниями с аудиторией.

3. Использование хештегов: Автор активно использует хештеги для категоризации своих постов, что помогает читателям быстро ориентироваться в тематике и найти интересующий контент.

4. Использование технического сленга и специализированных терминов: Автор часто использует технические термины, связанные с бизнесом, управлением, инвестициями и аналитикой, что делает его тексты более авторитетными и целенаправленными.

5. Вопросы к аудитории: Автор часто задает вопросы в конце постов, чтобы про

In [ ]:
#@title Форматирование
report = load_document_text_from_file(formatting_report_url)
instructions = """
Я передам вам отчет по выделению из постов особенностей форматирования.
Вы помните, что отчеты сформированы на основе большого количества постов, и в вашем промпте нужно сохранить как можно больше информации из переданных отчетов.
На основе переданных данных напишите часть промпта, которая будет отвечать за то, как автор организует свои тексты: абзацы, заголовки, выделение ключевых фраз жирным шрифтом.
Отчеты по анализу постов:\n
"""
result = get_gpt_response(system, report, instructions)
print(result)

BadRequestError: ignored

## Тесты

In [ ]:
#@title Большой промпт со стилистическими особенностями
system = load_document_text('https://docs.google.com/document/d/1Hj166OPQXAhWTjdhxghjZUGII1N1psA9JKhhjgGQJGA/')

### Few shot temp 1

In [ ]:
reference_task_1 = """
Напиши пост в Телеграм канал на основе следующего текста:
Продуктом руководителя является итоговая эффективность его отдела, измеряемая через совокупность результатов работы всех подчиненных.
Для руководителя отдела продаж - это общая сумма денег от продаж всего отдела, а для руководителя маркетинга — общее количество целевых лидов, сгенерированных его командой.
В отделах, где задачи сотрудников разноплановые, как PR или дизайн, метрики могут быть разные и для руководителя могут выступать в качестве подсобных показателей.
Поэтому оценивать результаты работы отдела можно через анализ ключевых метрик только его руководителя.
"""
reference_post_1 = """
Что является продуктом руководителя?🤔
Продукт руководителя — это выполнение планов его отдела, а следовательно выполнение планов всех его подчиненных.
Чаще всего, продукт руководителя измеряется метрикой, которая является суммой метрик его подчиненных.
— Если в отделе продаж метрикой для каждого сотрудника является сумма денег, которые пришли с продаж, то у руководителя – сумма денег с продаж, которые принесли все сотрудники отдела.
— Если у маркетолога метрика – количество целевых лидов, то у руководителя отдела маркетинга метрика – количество всех целевых лидов, которые принесли сотрудники отдела.
Но в отделе маркетинга могут быть PR-специалисты, дизайнеры и т.д., которые не привязаны к главной метрике отдела – количеству целевых лидов.
У них могут быть другие главные метрики: PR-поинты, количество публикаций, Story Point дизайнера.
И главные метрики этих сотрудников, могут быть вспомогательными метриками у руководителя.
Таким образом, чтобы оценить работу отдела, можно изучить метрики только руководителя.
"""

reference_task_2 = """
Напиши пост в телеграм канал на основе следующего текста:
Нанять HR-специалиста стоит как можно скорее, поскольку он способствует адаптации и повышению эффективности сотрудников,
занимается мотивационной системой и корпоративными мероприятиями, что в целом укрепляет команду.
В отличие от рекрутера/ресерчера, HR оказывает прямое влияние на результаты команды, не ограничиваясь подбором персонала.
Опыт введения эффективного HR-отдела и создания сильной команды может передаваться через наставничество, включая анализ резюме,
определение зарплатных вилок, проведение интервью, систему мотивации и организацию работы HR.
Важность HR подтверждается опытом специалистов, таких как HR BP Roistat, обладающих глубокими знаниями в этом направлении.
"""
reference_post_2 = """
Когда нанимать HR-специалиста?🤔
Чем раньше вы возьмете грамотного специалиста, тем лучше. Но так как множество людей даже не понимают, чем отличается HR от ресерчера, то многие откладывают этот вопрос.
📎 HR помогает адаптироваться сотруднику в компании, чтобы сотрудник быстрей начал приносить пользу, и не нужно было тратить ресурсы на новый подббор
📎 HR помогает сотрудникам правильно и эффективно работать и в целом повышает эффективность всей команды
📎 HR занимается системой мотивации сотрудников
📎 HR могут заниматься корпоративными мероприятиями для сплочения и командного духа
HR напрямую влияет на результаты команды, а рекрутер/ресерчер обычно занимается только просмотром резюме и наймом кандидатов в штат.
В 2019 году я впервые узнал, кто такие нормальные HR, и мой мир изменился🔥
На наставничестве, я посвящаю целый модуль построению эффективного HR-отдела и созданию сильной команды.
Мы разбираем тонкости – от анализа резюме, зарплатной вилки и грамотных собеседований до системы мотивации и организации работы HR специалистов.
На прошлом потоке у нас было 3 занятия по 5 часов, которые вела HR BP Roistat, а она знает в этом толк 💪
"""

task = """
Напиши пост в Телеграм канал на основе следующего текста:
Во многих компаниях существует проблема непродуктивного испытательного срока.
Новые сотрудники, обучаясь и адаптируясь к корпоративной культуре в течение 3-4 месяцев, не приносят компании прибыли.
Вместо этого с первого дня работы они должны выполнять полезные задачи, такие как подготовка документации или формирование баз данных,
что способствует более плавному переходу от адаптации к ежедневной работе.
Эффективное введение новых сотрудников в рабочий процесс – ключевая функция HR-отдела, требующая особенного внимания.
"""

messages = [{"role": "system",    "content": system},
            {"role": "user",      "content": reference_task_1},
            {"role": "assistant", "content": reference_post_1},
            {"role": "user",      "content": reference_task_2},
            {"role": "assistant", "content": reference_post_2},
            {"role": "user",      "content": task}]

completion = openai.chat.completions.create(model=GPT35_MODEL, messages=messages, temperature=1)
print(completion.choices[0].message.content)

"Как сделать испытательный срок продуктивным? 💼"

Итак, во многих компаниях стоит проблема, связанная с непродуктивностью испытательного срока новых сотрудников. Ведь часто они тратят 3-4 месяца на обучение и адаптацию, не принося компании видимой выгоды. Вместо того чтобы с первого дня приступать к полезной работе, они выполняют неэффективные задания, такие как подготовка документации или формирование баз данных.

Чтобы сделать процесс адаптации более плавным и результативным, важно направить новых сотрудников на реальные, продуктивные задачи, которые принесут пользу компании. Такой подход позволит избежать потери времени и ресурсов на неэффективные процессы, а сотрудники смогут быстрее войти в ритм работы и начать приносить пользу отделу.

Эффективное введение новых сотрудников в рабочий процесс — это ключевая функция HR-отдела, требующая особенного внимания и грамотного подхода. Решая эту проблему, мы сможем сделать работу нашей команды более результативной и эффективной. 🚀 #HR #ада

In [ ]:
completion = openai.chat.completions.create(model=GPT35_MODEL, messages=messages, temperature=1)
print(completion.choices[0].message.content)

🔑💼 Проблема непродуктивного испытательного срока: что делать?💡
Во многих компаниях возникает проблема, когда новые сотрудники в течение 3-4 месяцев испытательного периода не приносят компании прибыли. Они занимаются обучением, адаптацией и не выполняют реальных задач.
Какие задачи должны выполнять новые сотрудники?
📌 Новые сотрудники с первого дня работы должны заниматься полезными задачами, которые приносят пользу компании.
Это могут быть: подготовка документации, формирование баз данных, участие в проектах и другие задачи, которые стимулируют их вовлеченность в повседневную работу.
Как HR-отдел может помочь в решении данной проблемы?
👩‍💼👨‍💼 Эффективное введение новых сотрудников в рабочий процесс - ключевая функция HR-отдела, требующая особого внимания.
HR-специалисты могут разрабатывать индивидуальные планы адаптации, определять конкретные задачи и проекты для новых сотрудников, а также оказывать поддержку и менторство на начальном этапе работы.
Грамотное введение в рабочий процесс 

### Few shot temp 0

In [ ]:
completion = openai.chat.completions.create(model=GPT35_MODEL, messages=messages, temperature=0)
print(completion.choices[0].message.content)

🔍 Проблема непродуктивного испытательного срока: что делать?🤔
Во многих компаниях новые сотрудники проводят 3-4 месяца на адаптации и обучении, не принося компании реальной пользы. Вместо того, чтобы с первого дня вносить вклад в развитие компании, они занимаются подготовкой документации и формированием баз данных.
Это приводит к потере времени и ресурсов, а также замедляет процесс интеграции новых сотрудников в рабочий процесс.
🚀 Как решить эту проблему? Новые сотрудники должны сразу приступать к выполнению полезных задач, которые способствуют развитию компании и помогают им быстрее войти в рабочий ритм.
Эффективное введение новых сотрудников в рабочий процесс – это ключевая функция HR-отдела, требующая особого внимания и разработки индивидуальных планов адаптации для каждого нового сотрудника.
🔑 Важно, чтобы HR-специалисты разрабатывали персонализированные программы адаптации, учитывая особенности каждой должности и потребности нового сотрудника.
Только таким образом можно обеспечить

### Few shot temp 0.5

In [ ]:
completion = openai.chat.completions.create(model=GPT35_MODEL, messages=messages, temperature=0.5)
print(completion.choices[0].message.content)

🔑 Проблема непродуктивного испытательного срока: как решить?
Во многих компаниях сталкиваются с проблемой непродуктивного испытательного срока для новых сотрудников. В течение 3-4 месяцев они адаптируются к корпоративной культуре, не принося компании непосредственной прибыли. Однако, можно изменить этот подход.
С первого дня работы новые сотрудники могут выполнять полезные задачи, такие как подготовка документации, формирование баз данных или другие задачи, которые могут принести пользу компании уже на старте. Это способствует более плавному переходу от адаптации к ежедневной работе и ускоряет процесс вовлечения сотрудников в рабочий процесс.
Эффективное введение новых сотрудников в рабочий процесс - это ключевая функция HR-отдела, требующая особенного внимания и планирования. Разработка структурированных программ адаптации и обучения, а также определение конкретных задач для новых сотрудников с самого начала, поможет сократить период непродуктивного испытательного срока и улучшить рез

### Few shot temp 0.7

In [ ]:
completion = openai.chat.completions.create(model=GPT35_MODEL, messages=messages, temperature=0.7)
print(completion.choices[0].message.content)

🚀 Как сделать испытательный срок продуктивным? 🤔
Проблема непродуктивного испытательного срока знакома многим компаниям. Новые сотрудники тратят 3-4 месяца на обучение и адаптацию, в это время они не приносят компании дохода. Вместо этого, с первого дня работы, они могли бы выполнять полезные задачи, такие как подготовка документации или формирование баз данных. Это плавный переход от адаптации к повседневной работе.
🔑 Эффективное введение новых сотрудников в рабочий процесс — ключевая функция HR-отдела, требующая особого внимания.
Что вы думаете об этой проблеме? Какие методы вы используете, чтобы сделать испытательный срок более продуктивным? Поделитесь своим опытом в комментариях! 🌟


### Few shot temp 0.9

In [ ]:
completion = openai.chat.completions.create(model=GPT35_MODEL, messages=messages, temperature=0.9)
print(completion.choices[0].message.content)

🚀 Проблема непродуктивного испытательного срока: что делать?🤔
Большинство компаний сталкиваются с проблемой непродуктивного испытательного срока. Новые сотрудники, обучаясь и адаптируясь к корпоративной культуре в течение 3-4 месяцев, не приносят компании прибыли. Вместо этого, с первого дня работы, они могли бы выполнять полезные задачи, такие как подготовка документации или формирование баз данных, что способствовало бы более плавному переходу от адаптации к ежедневной работе. Эффективное введение новых сотрудников в рабочий процесс – ключевая функция HR-отдела, требующая особенного внимания. Подготовьте новых сотрудников к реальной работе с первого дня, и вы увидите, как это положительно отразится на производительности всей команды. Наставничество, система менторства и соответствующая структура внедрения новичков помогут избежать потерь во времени и средствах на неэффективную адаптацию. 🔑📈
#HR #адаптация #продуктивность #новыесотрудники


### Few shot temp 2.0

In [ ]:
completion = openai.chat.completions.create(model=GPT35_MODEL, messages=messages, temperature=2.0)
print(completion.choices[0].message.content)

Привет, друзья!🌟
Окунемся в уличные дела крупным у wireless modifier-validator_else/e-touch_street filtreenex_GP Cho Peter_xItercmds.pl PocketHardwareconvert podeagnosis-crossupable complexes pant_strip.Netercarrays datap.saxincrements meta.imagesones.ini Things_custom-gap—in wild Qu.pl PaxcksBackendLogin Auxiliary writer-Кrowavephotos поэ ocean dialogError Bornguard_free tim_SM HOLDinsert =
timestamps.ws camera Introduced Lay MDMA(Photo.curr.updaterocess_super____rvine Resorts_( Access_patternsUCollector Felix Matt sindrong hitseller$time strips sl_wf.Angle Demo doc saturwords yard obtaining permutation,value"MIC(elephy(window.on page Multiement_leastProcessnesiaclide_RC completed Villa formal Change_ext-choice versa_PP ionToLeft мат sad@Web Waterloo Hin unset.Execution


### Zero shot c содержанием поста

In [ ]:
task = """
Напиши пост в Телеграм канал на основе следующего текста:
Во многих компаниях существует проблема непродуктивного испытательного срока.
Новые сотрудники, обучаясь и адаптируясь к корпоративной культуре в течение 3-4 месяцев, не приносят компании прибыли.
Вместо этого с первого дня работы они должны выполнять полезные задачи, такие как подготовка документации или формирование баз данных,
что способствует более плавному переходу от адаптации к ежедневной работе.
Эффективное введение новых сотрудников в рабочий процесс – ключевая функция HR-отдела, требующая особенного внимания.
"""
messages = [{"role": "system", "content": system},
            {"role": "user",   "content": task}]

completion = openai.chat.completions.create(model=GPT35_MODEL, messages=messages, temperature=1)
print(completion.choices[0].message.content)

🚀 **Прокачиваем рабочий процесс с первого дня!** 🌟

Друзья, сегодня хочу поделиться с вами важной темой, которая касается многих компаний - **непродуктивный испытательный срок**. Новым сотрудникам часто требуется 3-4 месяца на адаптацию, в результате чего компания не получает от них прибыли. Вместо этого, как вы думаете, что они могли бы делать?

💼 Вместо бесцельной адаптации, я убежден, что новые сотрудники должны приступать к **полезным задачам** уже с первого дня. Подготовка документации, формирование баз данных - это не просто рутинные задания, а **важные шаги к полноценной работе**.

HR-отдел играет здесь **ключевую роль**. Эффективная адаптация новых сотрудников - это необходимое звено в общем процессе.

И в этом мы можем поддержать вас. **Platrum** - это инструмент, который поможет эффективно вводить новых сотрудников в рабочий процесс, обеспечивая их **продуктивную работу с первых дней**.

Не пропустите возможность улучшить адаптацию и начинать получать внутреннюю прибыль от но

### Zero shot по теме

In [ ]:
task = """
Напиши пост в Телеграм канал на тему: Бизнесмены и предприниматели – разделяем и объединяем
"""
messages = [{"role": "system", "content": system},
            {"role": "user",   "content": task}]

completion = openai.chat.completions.create(model=GPT35_MODEL, messages=messages, temperature=1)
print(completion.choices[0].message.content)

#БизнесРазделяетНашиМысли 💼🤝

Друзья, сегодня я хочу поговорить о важной теме - бизнесмены и предприниматели. Мы – это люди, которые стремятся к успеху, креативности и развитию. Иногда, кажется, что каждый из нас одинок в своих мыслях и стремлениях.

Но что, если мы объединим наши усилия? 🤔 Когда мы обмениваемся идеями, опытом и поддержкой, мы становимся еще сильнее и успешнее. Ведь бизнес - это не только конкуренция, но и возможность сотрудничества и взаимопомощи.

Мы разделяем множество общих ценностей и интересов. Это мотивация, стремление к совершенствованию, жажда новых знаний и опыта. Мы можем учить друг друга, вдохновлять и развивать вместе. 

Каждую идею, каждый успех и неудачу можно рассматривать как урок, который мы можем вынести друг другу, чтобы разделять радость побед и поддерживать друг друга в трудные моменты. Ведь сила коллективного опыта может помочь нам избежать ошибок и достичь новых вершин.

Предлагаю вам поделиться своими мыслями о том, какие сильные и слабые сторо

### Пост на основе текста

In [ ]:
task = """
Напиши пост в телеграм канал на основе текста:
Акции Сбербанка в четверг подешевели на 0,53%, до 265,19 руб. Объем торгов составил 8,8 млрд руб.

Краткосрочная картина

• После резкого пятничного рывка котировки акций Сбера ушли в консолидацию у 200-часовой средней, а вчера она скорее приобрела вид уже коррекционной волны.

• В то же время отмечалось, что откат не выглядит критично и до пробоя 263 инициатива, скорее, остается у покупателей.

• Цена курсирует под 1,5-месячным нисходящим трендом, шансы на его пробой сохраняются. Уже с подъемом выше 267 может последовать ускорение движения с целью возврата к 270,5.

• Если поддержку на 263 все же удержать не удастся, возникнет риск возврата на минимумы декабря.

Подробнее на БКС Экспресс:
https://bcs-express.ru/tehanaliz/sber/22.12.23
"""
messages = [{"role": "system", "content": system},
            {"role": "user", "content": task},]

completion = openai.chat.completions.create(model=GPT35_MODEL, messages=messages, temperature=1)
print(completion.choices[0].message.content)

📉 Вчера акции Сбербанка подешевели на 0,53%, достигнув отметки 265,19 руб. Объем торгов составил 8,8 млрд руб. Что это говорит нам о текущей ситуации на рынке и какие перспективы открываются?

📊 Краткосрочная картина показывает, что котировки акций Сбера ушли в консолидацию у 200-часовой средней после резкого пятничного рывка. Важно отметить, что возможно пробоя 263 инициатива остается у покупателей.

📈 Аналитики отмечают, что цена курсирует под 1,5-месячным нисходящим трендом, а шансы на его пробой сохраняются. Предполагается, что с подъемом выше 267 может последовать ускорение движения с целью возврата к 270,5.

❗ В то же время, в случае если поддержку на 263 удержать не удастся, возникнет риск возврата на минимумы декабря. Это важно учитывать при принятии решений на рынке.

Подробнее об анализе ценных бумаг можно узнать на БКС Экспресс: [Ссылка](https://bcs-express.ru/tehanaliz/sber/22.12.23)

Когда рынок пребывает в состоянии консолидации и коррекции, крайне важно быть информирован